In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
%aiida
import urllib.parse as urlparse
import ipywidgets as ipw
from IPython.display import display, clear_output
from aiidalab_widgets_base import viewer
import nglview
import numpy as np
import bokeh.plotting as bpl
import bokeh.models as bmd
from bokeh.io import output_notebook, show

In [ ]:
def plot(x, y, dev, title=None, xlabel=None, ylabel=None, clr_label=None, with_links=False):
    x = np.array(x)
    y = np.array(y)
    dev = np.array(dev)
    output_notebook(hide_banner=True)
    fig = bpl.figure(
        toolbar_location=None,
        title=title,
        x_axis_label=xlabel,
        y_axis_label=ylabel,
        tools=['tap', 'zoom_in', 'zoom_out', 'pan'], 
        output_backend='webgl',
    )
    
    source = bmd.ColumnDataSource(data=dict(x=x, y=y))
    source_dev = bmd.ColumnDataSource(data=dict(base=x, lower=y-dev, upper=y+dev))

    
    fig.line('x', 'y', source=source)
    fig.circle('x', 'y', size=10, fill_color="white", source=source)
    fig.add_layout(bmd.Whisker(source=source_dev, base="base", upper="upper", lower="lower"))
        
    show(fig)

In [ ]:
url = urlparse.urlsplit(jupyter_notebook_url)
uuid = urlparse.parse_qs(url.query)['uuid'][0]
workcalc = load_node(uuid=str(uuid))
ism = workcalc.outputs.output_parameters['isotherm']

isotherm = np.array([ism['pressure'], ism['loading_absolute_average'], ism['loading_absolute_dev']]).transpose()
ism_p, ism_load, ism_dev = zip(*isotherm)

enthalpy = np.array([ism['pressure'], ism['enthalpy_of_adsorption_average'], ism['enthalpy_of_adsorption_dev']]).transpose()
enth_p, enth_av, enth_dev = zip(*enthalpy)

molecule_converter = {
    "co2": "CO2",
    "ch4": "CH4",
    "n2" : "N2",
    "h2o": "H2O",
    "h2" : "H2",
    "o2" : "O2",
}

# Detailed Report

## Structure

In [ ]:
display(viewer(workcalc.inputs.structure))

## Isotherm plot

In [ ]:
plot(ism_p, ism_load, ism_dev,
     xlabel="Pressure [{}]".format(ism['pressure_unit']),
     ylabel="{} loading [{}]".format(molecule_converter[workcalc.inputs['molecule'].value],
                                     ism['loading_absolute_unit']))

## Enthalpy of adsorption plot

In [ ]:
plot(enth_p, enth_av, enth_dev,
     xlabel="Pressure [{}]".format(ism['pressure_unit']),
     ylabel="Enthalpy of adsorption [{}]".format(ism['enthalpy_of_adsorption_unit']))

## Computed properties

In [ ]:
display(viewer(workcalc.outputs.output_parameters))